<a href="https://colab.research.google.com/github/efedikmen/testrepo/blob/master/blackscholes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Calculating options prices and greeks using Black-Scholes for various European options
Contact ed1756@stern.nyu.edu for suggestions - WIP
\
\
*How to use the notebook*: 
1. Go to "Runtime" in the toolbar at the top left and select "run all".
2. Adjust the variables in the last cell according to the parameters of the option you want to value. Refer to the comments on each line for your inputs.
3. After entering your inputs, run the last cell by clicking on the run button that appears at the top left of the cell when you hover over the cell.
4. You will get the option price and a dictionary of first order greeks and gamma. The greeks are not yet adjusted in the forex and futures cases.
\
\
Black-Scholes pricing of the European call option for a stock without dividends is:
$$C=S\mathcal{N}(d_1)-Ke^{-Rt}\mathcal{N}(d_2)$$
where
$$d_1=\frac{\ln(\frac{S}{K})+Rt+\frac{\sigma^2t}{2}}{\sigma\sqrt{t}}$$
$$d_2=d_1-\sigma\sqrt{t}$$

- $S$ is the spot price of the underlying
- $K$ is the strike price
- $R$ is the annual risk free rate
- $t$ is time to maturity in years
- $\sigma$ is annualized volatility
- $\mathcal{N}()$ is the cdf of normal distribution

The put price with the same parameters can be calculated through put call parity, which should give the same answer with the below formula:
$$P=Ke^{-Rt}\mathcal{N}(-d_2)-S\mathcal{N}(-d_1)$$



In [ ]:
from scipy.stats import norm
import math

class Option:
    def __init__(self, call: bool, S: float, K: float, T: float,R: float,sigma:float):
        self.call=call
        self.S=S
        self.K=K
        self.t=T/365
        self.R=R
        self.sigma=sigma
        self.d1=(math.log(self.S/self.K)+R*self.t+(self.sigma**2*self.t/2))/(sigma*math.sqrt(self.t))
        self.d2= self.d1-self.sigma*math.sqrt(self.t)
        self.nd1=norm.cdf(self.d1)
        self.nd2=norm.cdf(self.d2)
        self.pdfd1=norm.pdf(self.d1)
    def price(self):
        callprice = self.S*self.nd1-self.K*math.exp(-self.R*self.t)*self.nd2
        if self.call:
            return callprice
        else:
            pvK=self.K/(1+self.R)**self.t
            return callprice+pvK-self.S
#Greeks
    def delta(self):
        if self.call:
            return self.nd1
        else:
            return self.nd1-1
    def gamma(self):
        return self.pdfd1/(self.S*self.sigma*math.sqrt(self.t))
    def theta(self):
        if self.call:
            return (-(self.S*self.pdfd1*self.sigma/(2*math.sqrt(self.t)))-self.R*self.K*math.exp(-self.R*self.t)*self.nd2)/365
        else:
            return (-(self.S*self.pdfd1*self.sigma/(2*math.sqrt(self.t)))+self.R*self.K*math.exp(-self.R*self.t)*norm.cdf(-self.d2))/365
    def vega(self):
        return self.S*math.sqrt(self.t)*self.pdfd1*0.01
    def rho(self):
        if self.call:
            return self.K*self.t*math.exp(-self.R*self.t)*self.nd2*0.01
        else:
            return -self.K*self.t*math.exp(-self.R*self.t)*norm.cdf(-self.d2)*0.01
    def greeks(self):
        return {"delta":self.delta(),"gamma":self.gamma(),"theta":self.theta(),"vega":self.vega(),"rho":self.rho()}
    

When the firm pays a set dividend, we denote the annualized dividend yield by $q$. Then the call price of a dividend paying stock is:
$$C=Se^{-qt}\mathcal{N}(d_1)-Ke^{-Rt}\mathcal{N}(d_2)$$
where
$$d_1=\frac{\ln(\frac{S}{K})+(R-q)t+\frac{\sigma^2t}{2}}{\sigma\sqrt{t}}$$
$$d_2=d_1-\sigma\sqrt{t}$$

Likewise, the put price is:
$$P=Ke^{-Rt}\mathcal{N}(-d_2)-Se^{-qt}\mathcal{N}(-d_1)$$


In [ ]:
class DividendOption(Option):
    def __init__(self,call: bool, S: float, K: float, T: float,R: float,q:float,sigma:float):
        super().__init__(call,S,K,T,r,sigma)
        self.q=q
        self.d1=(math.log(S/K)+(R-q)*t+(sigma**2*t/2))/(sigma*math.sqrt(t))
    
    def price(self):
        if self.call:
            return self.S*math.exp(-self.q*self.t)*self.nd1-self.K*math.exp(-self.R*self.t)*self.nd2
        else:
            return self.K*math.exp(-self.R*self.t)*norm.cdf(-self.d2)-self.S*math.exp(-self.q*self.t)*norm.cdf(-self.d1)
    #Greeks
    def delta(self):
        if self.call:
            return math.exp(-self.q*self.t)*self.nd1
        else:
            return math.exp(-self.q*self.t)*(1-self.nd1)
    def gamma(self):
        return self.pdfd1*math.exp(-self.q*self.t)/(self.S*self.sigma*math.sqrt(self.t))
    def theta(self):
        first= self.S*self.pdfd1*self.sigma*math.exp(-self.q*self.t)/(2*math.sqrt(self.t))
        second=self.q*self.S*math.exp(-self.q*self.t)
        third=self.R*self.K*math.exp(-self.R*self.t)
        if self.call:
            return (-first+second*self.nd1-third*self.nd2)/365
        else:
            return (-first-second*norm.cdf(-self.d1)+third*norm.cdf(-self.d2))/365
    def vega(self):
        return self.S*math.sqrt(self.t)*self.pdfd1*math.exp(-self.q*self.t)*0.01
    def rho(self):
        if self.call:
            return self.K*self.t*math.exp(-self.R*self.t)*self.nd2*0.01
        else:
            return -self.K*self.t*math.exp(-self.R*self.t)*norm.cdf(-self.d2)*0.01
        

For a European futures option, we have that 
$$S=Fe^{-Rt}$$
where $F$ is the spot futures price. Then we derive the call price such that:
$$C=e^{-Rt}(F\mathcal{N}(d_1)-K\mathcal{N}(d_2))$$
where
$$d_1=\frac{\ln(\frac{F}{K})+\frac{\sigma^2t}{2}}{\sigma\sqrt{t}}$$
$$d_2=d_1-\sigma\sqrt{t}$$
Note that $\sigma$ is the annual volatility of the futures price. The price of the put follows:
$$P=e^{-Rt}(K\mathcal{N}(-d_2)-F\mathcal{N}(-d_1))$$

In [ ]:
class FuturesOption(Option):
    def __init__(self, call: bool, S: float, K: float, T: float,R: float,sigma:float):
        super().__init__(call,S,K,T,R,sigma)
        self.d1=(math.exp(self.S/self.K)+self.sigma**2*self.t/2)/(self.sigma*math.sqrt(self.t))
    def price(self):
        if self.call:
            return math.exp(-self.R*self.t)*((self.S*self.nd1)-self.K*self.nd2)
        else:
            return math.exp(-self.R*self.t)*((self.K*norm.cdf(-self.d2))-self.S*norm.cdf(-self.d1))

European call option on forex. Take a call option on a foreign currency, which will give the right to buy the foreign currency with the domestic currency at the strike price. Then, we denote $R_f$ as the annual risk free rate in the foreign country and $R$ the annual domestic risk free rate. The call price is:
$$C=Se^{-R_ft}\mathcal{N}(d_1)-Ke^{-Rt}\mathcal{N}(d_2)$$
where
$$d_1=\frac{\ln(\frac{S}{K})+(R-R_f)t+\frac{\sigma^2t}{2}}{\sigma\sqrt{t}}$$
$$d_2=d_1-\sigma\sqrt{t}$$
Note that $\sigma$ is the annual volatility of the futures price. The price of the put follows:
$$P=Ke^{-Rt}\mathcal{N}(-d_2)-Se^{-R_ft}\mathcal{N}(-d_1)$$

In [ ]:
class ForexOption(Option):
    def __init__(self, call: bool, S: float, K: float, T: float,R: float,Rf:float,sigma:float):
        super().__init__(call,S,K,T,r,sigma)
        self.Rf=Rf
        self.d1=(math.exp(self.S/self.K)+(self.R-self.Rf)*self.t+self.sigma**2*self.t/2)/(self.sigma*math.sqrt(self.t))
    def price(self):
        if self.call:
            return self.S*math.exp(-self.Rf*self.t)*self.nd1-self.K*math.exp(-self.R*self.t)*self.nd2
        else:
            return self.K*math.exp(-self.R*self.t)*norm.cdf(-self.d2)-self.S*math.exp(-self.Rf*self.t)*norm.cdf(-self.d1)

In [ ]:
futures= False #enter true if it is a futures option
call= True #enter True for call, False for put
S= 100 #spot price, enter exchange rate for forex and futures price for futures options
K= 100 #strike
t= 30 #days to maturity
r= 0.01#annualized risk free rate, 0.01=1%
rf= 0 #annualized risk free rate for the foreign currency, leave as 0 unless for an forex option, 0.01=1%
q= 0 #annual dividend rate, leave as 0 unless for an option on a dividend paying stock, 0.01=1%
sigma= 0.5 #annualized volatility in the price of the underlying, 0.01=1%

if q!=0:
    opt=DividendOption(call,S,K,t,r,q,sigma)
    print(opt.price())
    print(opt.greeks())
elif rf!=0:
    opt=ForexOption(call,S,K,t,r,rf,sigma)
    print(opt.price())
elif futures:
    opt=FuturesOption(call,S,K,t,r,sigma)
    print(opt.price())
else:
    opt=Option(call,S,K,t,r,sigma)
    print(opt.price())
    print(opt.greeks())

5.752593081943331
{'delta': 0.530849952434708, 'gamma': 0.027747557988299164, 'theta': -0.09632266166298419, 'vega': 0.11403106022588698, 'rho': 0.03890334424235134}
